In [ ]:
!pip install accelerate datasets seqeval evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_name = "sbunlp/fabert"
twiner = load_dataset('Khedesh/ParsTwiNER')
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenized_twiner = twiner.map(tokenize_and_align_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/6418 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/447 [00:00<?, ? examples/s]

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
label_list = twiner["train"].features[f"ner_tags"].feature.names

id2label = {i:x for i,x in enumerate(label_list)}
label2id = {x:i for i,x in enumerate(label_list)}

In [ ]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), id2label=id2label, label2id=label2id)


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir = "/tmp/parstwiner",
    learning_rate= 2e-5,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size= 8,
    num_train_epochs= 3,
    weight_decay= 0,
    warmup_ratio = 0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_twiner["train"],
    eval_dataset=tokenized_twiner["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    )
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.377900,0.095227,0.783486,0.839725,0.810631,0.973136
2,0.076300,0.080997,0.814815,0.865290,0.839294,0.977088
3,0.053700,0.090036,0.810613,0.871190,0.839810,0.977624


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=2409, training_loss=0.13763463521904348, metrics={'train_runtime': 344.1728, 'train_samples_per_second': 55.943, 'train_steps_per_second': 6.999, 'total_flos': 652247274901368.0, 'train_loss': 0.13763463521904348, 'epoch': 3.0})

In [ ]:
validation_results = trainer.evaluate()

In [ ]:
print(validation_results)

{'eval_loss': 0.09003618359565735, 'eval_precision': 0.8106129917657823, 'eval_recall': 0.8711897738446411, 'eval_f1': 0.8398104265402845, 'eval_accuracy': 0.9776244389361559, 'eval_runtime': 2.4071, 'eval_samples_per_second': 185.7, 'eval_steps_per_second': 23.264, 'epoch': 3.0}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_twiner["train"],
    eval_dataset=tokenized_twiner["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [ ]:
print(test_results)

{'eval_loss': 0.08491431176662445, 'eval_precision': 0.7994310099573257, 'eval_recall': 0.8515151515151516, 'eval_f1': 0.8246515040352165, 'eval_accuracy': 0.9786, 'eval_runtime': 1.8823, 'eval_samples_per_second': 161.507, 'eval_steps_per_second': 20.188}


In [ ]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



         EVE     0.5294    0.6429    0.5806        14

         LOC     0.8465    0.8733    0.8597       221

         NAT     0.6429    0.9000    0.7500        30

         ORG     0.6619    0.7132    0.6866       129

         PER     0.8972    0.9303    0.9135       244

         POG     0.5833    0.6364    0.6087        22



   micro avg     0.7994    0.8515    0.8247       660

   macro avg     0.6935    0.7827    0.7332       660

weighted avg     0.8044    0.8515    0.8265       660

